# 样例介绍
* YOLOv5是一种单阶段目标检测算法，在这个样例中，我们选取了YOLOv5s，它是YOLOv5系列中较为轻量的网络，适合在边缘设备部署，进行实时目标检测。

# 前期准备
* 基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可以参考下面的步骤。
* 首先我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，下载我们已经准备好了的ONNX模型，ONNX是开源的离线推理模型框架。

* 为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用，以下为转换指令：  
    ```shell
    atc --model=yolov5s.onnx --framework=5 --output=yolo --input_format=NCHW --input_shape="input_image:1,3,640,640" --log=error --soc_version=Ascend310B1
    ```
    * 其中转换参数的含义为：  
        * --model：输入模型路径
        * --framework：原始网络模型框架类型，5表示ONNX
        * --output：输出模型路径
        * --input_format：输入Tensor的内存排列方式
        * --input_shape：指定模型输入数据的shape
        * --log：日志级别
        * --soc_version：昇腾AI处理器型号
        * --input_fp16_nodes：指定输入数据类型为FP16的输入节点名称
        * --output_type：指定网络输出数据类型或指定某个输出节点的输出类型

# 模型推理实现

In [1]:
# 导入代码依赖
import cv2
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import torch
from sklearn.cluster import KMeans
from skvideo.io import vreader, FFmpegWriter
import IPython.display
from ais_bench.infer.interface import InferSession

from det_utils import letterbox, scale_coords, nms

/usr/local/miniconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def preprocess_image(image, cfg, bgr2rgb=True):
    """图片预处理"""
    img, scale_ratio, pad_size = letterbox(image, new_shape=cfg['input_shape'])
    if bgr2rgb:
        img = img[:, :, ::-1]
    img = img.transpose(2, 0, 1)  # HWC2CHW
    img = np.ascontiguousarray(img, dtype=np.float32)/255
    return img, scale_ratio, pad_size


def draw_bbox(bbox, img0, color, wt, names, num, a, b, c, d):
    """在图片上画预测框"""
    det_result_str = ''
    for idx, class_id in enumerate(bbox[:, 5]):
        if float(bbox[idx][4] < float(0.05)):
            continue
        img0 = cv2.rectangle(img0, (int(bbox[idx][0]), int(bbox[idx][1])), (int(bbox[idx][2]), int(bbox[idx][3])),
                             color, wt)
        img0 = cv2.putText(img0, str(idx) + ' ' + names[int(class_id)], (int(bbox[idx][0]), int(bbox[idx][1] + 16)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        img0 = cv2.putText(img0, '{:.4f}'.format(bbox[idx][4]), (int(bbox[idx][0]), int(bbox[idx][1] + 32)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        det_result_str += '{} {} {} {} {} {}\n'.format(
            names[bbox[idx][5]], str(bbox[idx][4]), bbox[idx][0], bbox[idx][1], bbox[idx][2], bbox[idx][3])
    # 在图像的右下角添加文本
    text = f"total:{num}  8nm:{a}  10nm:{b}  12nm:{c}  14nm:{d}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.45
    font_thickness = 1
    font_color = (0, 0, 255)  # 红色
    # 获取文本的大小
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    # 计算文本放置的位置（右下角）
    text_position = (10, img0.shape[0] - 10)
    # 使用cv2.putText在图像上绘制文本
    cv2.putText(img0, text, text_position, font, font_scale, font_color, font_thickness)
    return img0


def get_labels_from_txt(path):
    """从txt文件获取图片标签"""
    labels_dict = dict()
    with open(path) as f:
        for cat_id, label in enumerate(f.readlines()):
            labels_dict[cat_id] = label.strip()
    return labels_dict


def draw_prediction(pred, image, labels, num, a, b, c, d):
    """在图片上画出预测框并进行可视化展示"""
    imgbox = widgets.Image(format='jpg', height=720, width=1280)
    img_dw = draw_bbox(pred, image, (0, 255, 0), 2, labels, num, a, b, c, d)
    imgbox.value = cv2.imencode('.jpg', img_dw)[1].tobytes()
    display(imgbox)
    
def calculate_num_c(pred):
    areas=[]
    rows, columns = pred.shape
    for i in range(rows):
        area=(pred[i,2]-pred[i,0])*(pred[i,3]-pred[i,1])
        areas.append(area)
    if(len(areas)>=4):
        areas_array = np.array(areas).reshape(-1, 1)  # 转换成二维数组，-1表示自动推断行数，1表示1列
        # 使用K均值聚类，假设你希望将面积分成4个类别
        num_clusters = 4
        kmeans = KMeans(n_clusters=num_clusters)
        kmeans.fit(areas_array)
        # 获取聚类结果
        cluster_labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        dic={0:int(centroids[0]),1:int(centroids[1]),2:int(centroids[2]),3:int(centroids[3])}
        new_sys2 = sorted(dic.items(), key=lambda d: d[1], reverse=False)
        # 打印每个框的面积和所属类别
        cnt=[0,0,0,0]
        for i, area in enumerate(areas):
            if((int)(cluster_labels[i])==0):
                cnt[0]+=1
            elif((int)(cluster_labels[i])==1):
                cnt[1]+=1
            elif((int)(cluster_labels[i])==2):
                cnt[2]+=1
            elif((int)(cluster_labels[i])==3):
                cnt[3]+=1
    else:
        return 0,0,0,0,0
    return len(areas),cnt[new_sys2[0][0]],cnt[new_sys2[1][0]],cnt[new_sys2[2][0]],cnt[new_sys2[3][0]]

def infer_image(img_path, model, class_names, cfg):
    """图片推理"""
    # 图片载入
    image = cv2.imread(img_path)
    # 数据预处理
    img, scale_ratio, pad_size = preprocess_image(image, cfg)
    # 模型推理
    output = model.infer([img])[0]

    output = torch.tensor(output)
    # 非极大值抑制后处理
    boxout = nms(output, conf_thres=cfg["conf_thres"], iou_thres=cfg["iou_thres"])
    pred_all = boxout[0].numpy()
    # 计算小球数量
    num,a,b,c,d=calculate_num_c(pred_all)
    # 预测坐标转换
    scale_coords(cfg['input_shape'], pred_all[:, :4], image.shape, ratio_pad=(scale_ratio, pad_size))
    # 图片预测结果可视化
    draw_prediction(pred_all, image, class_names, num, a, b, c, d)


def infer_frame_with_vis(image, model, labels_dict, cfg, bgr2rgb=True):
    # 数据预处理
    img, scale_ratio, pad_size = preprocess_image(image, cfg, bgr2rgb)
    # 模型推理
    output = model.infer([img])[0]

    output = torch.tensor(output)
    # 非极大值抑制后处理
    boxout = nms(output, conf_thres=cfg["conf_thres"], iou_thres=cfg["iou_thres"])
    pred_all = boxout[0].numpy()
    # 计算小球数量
    num,a,b,c,d=calculate_num_c(pred_all)
    # 预测坐标转换
    scale_coords(cfg['input_shape'], pred_all[:, :4], image.shape, ratio_pad=(scale_ratio, pad_size))
    # 图片预测结果可视化
    img_vis = draw_bbox(pred_all, image, (0, 255, 0), 2, labels_dict, num, a, b, c, d)
    return img_vis


def img2bytes(image):
    """将图片转换为字节码"""
    return bytes(cv2.imencode('.jpg', image)[1])


def infer_video(video_path, model, labels_dict, cfg):
    """视频推理"""
    image_widget = widgets.Image(format='jpeg', width=800, height=600)
    display(image_widget)

    # 读入视频
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, img_frame = cap.read()
        if not ret:
            break
        # 对视频帧进行推理
        image_pred = infer_frame_with_vis(img_frame, model, labels_dict, cfg, bgr2rgb=True)
        image_widget.value = img2bytes(image_pred)


def infer_camera(model, labels_dict, cfg):
    """外设摄像头实时推理"""
    def find_camera_index():
        max_index_to_check = 10  # Maximum index to check for camera

        for index in range(max_index_to_check):
            cap = cv2.VideoCapture(index)
            if cap.read()[0]:
                cap.release()
                return index

        # If no camera is found
        raise ValueError("No camera found.")

    # 获取摄像头
    camera_index = find_camera_index()
    cap = cv2.VideoCapture(camera_index)
    # 初始化可视化对象
    image_widget = widgets.Image(format='jpeg', width=1280, height=720)
    display(image_widget)
    while True:
        start=time.time()
        # 对摄像头每一帧进行推理和可视化
        _, img_frame = cap.read()
        image_pred = infer_frame_with_vis(img_frame, model, labels_dict, cfg)
        image_widget.value = img2bytes(image_pred)
        end=time.time()
        print(end-start)

# 样例运行

* 初始化相关参数

In [3]:
cfg = {
    'conf_thres': 0.2,  # 模型置信度阈值，阈值越低，得到的预测框越多
    'iou_thres': 0.3,  # IOU阈值，高于这个阈值的重叠预测框会被过滤掉
    'input_shape': [640, 640],  # 模型输入尺寸
}

model_path = 'best.om'
label_path = './test.txt'
# 初始化推理模型
model = InferSession(0, model_path)
labels_dict = get_labels_from_txt(label_path)

* 选择推理模式。"infer_mode"有三个取值：image, camera, video，分别对应图片推理、摄像头实时推理和视频推理。默认使用视频推理模式。
* 我们选取的样例是一个赛车视频，执行下面的代码后可以看到模型会对视频的每一帧进行推理，并将预测结果展示在画面上。

In [ ]:
infer_mode = 'camera'

if infer_mode == 'image':
    img_path = 'test2.jpg'
    infer_image(img_path, model, labels_dict, cfg)
elif infer_mode == 'camera':
    infer_camera(model, labels_dict, cfg)
elif infer_mode == 'video':
    video_path = 'racing.mp4'
    infer_video(video_path, model, labels_dict, cfg)

[INFO] acl init success
[INFO] open device 0 success
[INFO] load model best.om success
[INFO] create model description success


[ WARN:0@7.963] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@8.148] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


Image(value=b'', format='jpeg', height='720', width='1280')

0.4172341823577881
0.17367172241210938
0.16678285598754883
0.16647863388061523
0.16730809211730957
0.16872429847717285
0.16738128662109375
0.16556906700134277
0.16688871383666992
0.1697676181793213
0.17064666748046875
0.16693496704101562
0.16608762741088867
0.1647963523864746
0.16610026359558105
0.16544342041015625
0.16594171524047852
0.16531729698181152
0.16605186462402344
0.16508841514587402
0.16755914688110352
0.16712069511413574
0.16747832298278809
0.16881489753723145
0.16720056533813477
0.165604829788208
0.1678910255432129
0.16700243949890137
0.1667191982269287
0.16621851921081543
0.16688108444213867
0.16593718528747559
0.16701340675354004
0.16673779487609863
0.16783356666564941
0.16500568389892578
0.16617107391357422
0.16685962677001953
0.16978240013122559
0.16916465759277344
0.167525053024292
0.16599106788635254
0.1679084300994873
0.16935491561889648
0.1694793701171875
0.16570329666137695
0.16900205612182617
0.1656358242034912
0.16661906242370605
0.16706228256225586
0.1680326461

/usr/local/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

0.22299814224243164


/usr/local/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.21515536308288574
0.1753993034362793
0.17010784149169922
0.16842389106750488
0.1681528091430664
0.16905546188354492


/usr/local/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.22110342979431152
0.17129945755004883
0.16658806800842285
0.16789889335632324
0.16984820365905762
0.16666650772094727
0.16849708557128906
0.16548466682434082
0.16485953330993652
0.16448569297790527
0.16903352737426758
0.16459417343139648
0.1654987335205078
0.164473295211792
0.16530346870422363
0.16633009910583496
0.1687164306640625
0.16829681396484375
0.17023491859436035
0.16659760475158691
0.16770052909851074
0.16434383392333984
0.1724076271057129
0.16865134239196777
0.17109322547912598
0.16675806045532227
0.16820430755615234
0.16687965393066406
0.16750717163085938
0.16716623306274414
0.1683042049407959
0.16742229461669922
0.1697831153869629
0.1694803237915039
0.16895723342895508
0.16666364669799805
0.171067476272583
0.1671745777130127
0.16805720329284668
0.16571378707885742
0.17280220985412598
0.16786456108093262
0.16907453536987305
0.16616344451904297
0.16919946670532227
0.1676168441772461
0.16505861282348633
0.16489076614379883
0.16822099685668945
0.1672382354736328
0.16526865959

# 样例总结与扩展
以上就是这个样例的全部内容了，值得关注的是在模型推理后有一步非常重要的后处理，就是非极大值抑制，即NMS，由于模型的原始预测结果会有非常多无效或重叠的预测框，我们需要通过NMS来进行过滤。再者，模型预测框的表示往往是一个标准化的结果，比如0到1之间，我们需要通过坐标转换将结果与原始图片的宽高对应上。